In [35]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords 
from nltk.tokenize import WordPunctTokenizer

In [36]:
import warnings
warnings.filterwarnings('ignore')

In [37]:
df = pd.read_csv('mod_data2.txt', encoding = "ISO-8859-1")

In [38]:
df.head()

business_id        date               review_id  stars  \
0  9yKzy9PApeiPPOUJEtnvkg  2011-01-26  fWKvX83p0-ka4JS3dc6E5A      5   
1  ZRJwVLyzEJq1VAihDhYiow  2011-07-27  IjZ33sJrzXqU-0X6U8NwyA      5   
2  6oRAC4uyJCsJl1X0WZpVSA  2012-06-14  IESLBzqUCLdSzSqm0eCSxQ      4   
3  _1QQZuf4zZOyFCvXc0o6Vg  2010-05-27  G-WvGaISbqqaMHlNnByodA      5   
4  6ozycU1RpktNG2-1BroVtw  2012-01-05  1uJFq2r5QfJG_6ExMRCaGw      5   

                                                text    type  \
0  My wife took me here on my birthday for breakf...  review   
1  I have no idea why some people give bad review...  review   
2  love the gyro plate.]Rice is so good and I als...  review   
3  Rosie, Dakota, and I LOVE Chaparral Dog Park!!...  review   
4  General Manager Scott Petello is a good egg!!!...  review   

                  user_id  cool  useful  funny  
0  rLtl8ZkDX5vH5nAx9C3q5Q     2       5      0  
1  0a2KyEL0d3Yb1V6aivbIuQ     0       0      0  
2  0hT2KtfLiobPvh6cDC8JQg     0       1      0  
3  uZetl9T0NcROGOyFfughhg     1       2      0  
4  vYmM4KTsC8ZfQBg-j5MWkw     0       0      0

In [39]:
import string
import re
def clean_text(text):
    text = text.translate(string.punctuation)
    
    ## Convert words to lower case and split them
    text = text.lower().split()
    
    ## Remove stop words
    stops = set(stopwords.words("english"))
    text = [w for w in text if not w in stops and len(w) >= 3]
    
    text = " ".join(text)
    
    # Clean the text
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)    
    return text

In [40]:
yelp_data = df[['business_id', 'user_id', 'stars', 'text']]

In [41]:
%%time
yelp_data['text'] = yelp_data['text'].apply(clean_text)

Wall time: 317 ms


In [42]:
#Split train test
vld_size=0.15
X_train, X_valid, y_train, y_valid = train_test_split(yelp_data['text'], df['business_id'], test_size = vld_size) 

In [43]:
userid_df = yelp_data[['user_id','text']]
business_df = yelp_data[['business_id', 'text']]

In [44]:
userid_df = userid_df.groupby('user_id').agg({'text': ' '.join})
business_df = business_df.groupby('business_id').agg({'text': ' '.join})

In [45]:
#userid vectorizer
userid_vectorizer = TfidfVectorizer(tokenizer = WordPunctTokenizer().tokenize, max_features=1000)
userid_vectors = userid_vectorizer.fit_transform(userid_df['text'])
userid_vectors.shape

(262, 1000)

In [46]:
#Business id vectorizer
businessid_vectorizer = TfidfVectorizer(tokenizer = WordPunctTokenizer().tokenize, max_features=1000)
businessid_vectors = businessid_vectorizer.fit_transform(business_df['text'])
businessid_vectors.shape

(253, 1000)

# Matrix Factorization

In [47]:
userid_rating_matrix = pd.pivot_table(yelp_data, values='stars', index=['user_id'], columns=['business_id'])
userid_rating_matrix.shape

(262, 253)

In [48]:
userid_rating_matrix.head()

business_id             -0bUDim5OGuv8R0Qqq6J4A  -kVXDEqGHOWKxQ3EhvFgVA  \
user_id                                                                  
-OMlS6yWkYjVldNhC31wYg                     NaN                     NaN   
-V3CXSxnUzjkLVqi1xGrkA                     NaN                     NaN   
-Wa5j14Rhps3DC2SMaaatQ                     NaN                     NaN   
-yg_AMAU2HNh48zcuQ13mw                     NaN                     NaN   
0CMz8YaO3f8xu4KqQgKb9Q                     NaN                     NaN   

business_id             -yxfBYGB6SEqszmxJxd97A  06kfoeRs9Acj82Yl3i9p_w  \
user_id                                                                  
-OMlS6yWkYjVldNhC31wYg                     NaN                     NaN   
-V3CXSxnUzjkLVqi1xGrkA                     NaN                     NaN   
-Wa5j14Rhps3DC2SMaaatQ                     NaN                     NaN   
-yg_AMAU2HNh48zcuQ13mw                     NaN                     NaN   
0CMz8YaO3f8xu4KqQgKb9Q                     NaN                     NaN   

business_id             08Z_Zzp8PyEmWWpYurIO-Q  0BpMvu5B9fY-KEbOuxLtFQ  \
user_id                                                                  
-OMlS6yWkYjVldNhC31wYg                     NaN                     NaN   
-V3CXSxnUzjkLVqi1xGrkA                     NaN                     NaN   
-Wa5j14Rhps3DC2SMaaatQ                     NaN                     NaN   
-yg_AMAU2HNh48zcuQ13mw                     NaN                     NaN   
0CMz8YaO3f8xu4KqQgKb9Q                     NaN                     NaN   

business_id             0QTn4pMzKv1mnQifcP9YoQ  0QrA-Klgp1R-GzUr6uJS7Q  \
user_id                                                                  
-OMlS6yWkYjVldNhC31wYg                     NaN                     NaN   
-V3CXSxnUzjkLVqi1xGrkA                     NaN                     NaN   
-Wa5j14Rhps3DC2SMaaatQ                     NaN                     NaN   
-yg_AMAU2HNh48zcuQ13mw                     NaN                     NaN   
0CMz8YaO3f8xu4KqQgKb9Q                     NaN                     NaN   

business_id             0RqNRc6RiILzZxK9J8Kqug  145SQ_msdQ1yo7J0E63VoA  \
user_id                                                                  
-OMlS6yWkYjVldNhC31wYg                     NaN                     NaN   
-V3CXSxnUzjkLVqi1xGrkA                     NaN                     NaN   
-Wa5j14Rhps3DC2SMaaatQ                     NaN                     NaN   
-yg_AMAU2HNh48zcuQ13mw                     NaN                     NaN   
0CMz8YaO3f8xu4KqQgKb9Q                     NaN                     NaN   

business_id                      ...            xmtKVO7C7KYVqMBe2eQq3A  \
user_id                          ...                                     
-OMlS6yWkYjVldNhC31wYg           ...                               NaN   
-V3CXSxnUzjkLVqi1xGrkA           ...                               NaN   
-Wa5j14Rhps3DC2SMaaatQ           ...                               NaN   
-yg_AMAU2HNh48zcuQ13mw           ...                               NaN   
0CMz8YaO3f8xu4KqQgKb9Q           ...                               NaN   

business_id             y6uO4ydAwBHUujfiSktxZg  yGmdo1ENajB98iryHGoWFw  \
user_id                                                                  
-OMlS6yWkYjVldNhC31wYg                     NaN                     NaN   
-V3CXSxnUzjkLVqi1xGrkA                     NaN                     NaN   
-Wa5j14Rhps3DC2SMaaatQ                     NaN                     NaN   
-yg_AMAU2HNh48zcuQ13mw                     NaN                     NaN   
0CMz8YaO3f8xu4KqQgKb9Q                     NaN                     NaN   

business_id             yJr24Yy1K6bt2G9fX3_zPA  yPJFfglhMHAKciUpjTmgBg  \
user_id                                                                  
-OMlS6yWkYjVldNhC31wYg                     NaN                     NaN   
-V3CXSxnUzjkLVqi1xGrkA                     NaN                     NaN   
-Wa5j14Rhps3DC2SMaaatQ                     NaN                     NaN 

In [49]:
P = pd.DataFrame(userid_vectors.toarray(), index=userid_df.index, columns=userid_vectorizer.get_feature_names())
Q = pd.DataFrame(businessid_vectors.toarray(), index=business_df.index, columns=businessid_vectorizer.get_feature_names())


In [50]:
Q.head()

!         +         -    1   10   15    2   20  \
business_id                                                                     
-0bUDim5OGuv8R0Qqq6J4A  0.000000  0.000000  0.000000  0.0  0.0  0.0  0.0  0.0   
-kVXDEqGHOWKxQ3EhvFgVA  0.000000  0.000000  0.085288  0.0  0.0  0.0  0.0  0.0   
-yxfBYGB6SEqszmxJxd97A  0.000000  0.214001  0.000000  0.0  0.0  0.0  0.0  0.0   
06kfoeRs9Acj82Yl3i9p_w  0.000000  0.000000  0.083387  0.0  0.0  0.0  0.0  0.0   
08Z_Zzp8PyEmWWpYurIO-Q  0.219165  0.570900  0.000000  0.0  0.0  0.0  0.0  0.0   

                          3   30  ...   yellow  yelp  yes  yesterday  \
business_id                       ...                                  
-0bUDim5OGuv8R0Qqq6J4A  0.0  0.0  ...      0.0   0.0  0.0        0.0   
-kVXDEqGHOWKxQ3EhvFgVA  0.0  0.0  ...      0.0   0.0  0.0        0.0   
-yxfBYGB6SEqszmxJxd97A  0.0  0.0  ...      0.0   0.0  0.0        0.0   
06kfoeRs9Acj82Yl3i9p_w  0.0  0.0  ...      0.0   0.0  0.0        0.0   
08Z_Zzp8PyEmWWpYurIO-Q  0.0  0.0  ...      0.0   0.0  0.0        0.0   

                             yet  yoga  you       yum  yummy  zach  
business_id                                                         
-0bUDim5OGuv8R0Qqq6J4A  0.000000   0.0  0.0  0.000000    0.0   0.0  
-kVXDEqGHOWKxQ3EhvFgVA  0.000000   0.0  0.0  0.000000    0.0   0.0  
-yxfBYGB6SEqszmxJxd97A  0.069083   0.0  0.0  0.074398    0.0   0.0  
06kfoeRs9Acj82Yl3i9p_w  0.000000   0.0  0.0  0.000000    0.0   0.0  
08Z_Zzp8PyEmWWpYurIO-Q  0.000000   0.0  0.0  0.000000    0.0   0.0  

[5 rows x 1000 columns]

In [51]:
def matrix_factorization(R, P, Q, steps=1, gamma=0.001,lamda=0.02):
    for step in range(steps):
        for i in R.index:
            for j in R.columns:
                if R.loc[i,j]>0:
                    eij=R.loc[i,j]-np.dot(P.loc[i],Q.loc[j])
                    P.loc[i]=P.loc[i]+gamma*(eij*Q.loc[j]-lamda*P.loc[i])
                    Q.loc[j]=Q.loc[j]+gamma*(eij*P.loc[i]-lamda*Q.loc[j])
        e=0
        for i in R.index:
            for j in R.columns:
                if R.loc[i,j]>0:
                    e= e + pow(R.loc[i,j]-np.dot(P.loc[i],Q.loc[j]),2)+lamda*(pow(np.linalg.norm(P.loc[i]),2)+pow(np.linalg.norm(Q.loc[j]),2))
        if e<0.001:
            break
        
    return P,Q


In [52]:
%%time
P, Q = matrix_factorization(userid_rating_matrix, P, Q, steps=1, gamma=0.001,lamda=0.02)

Wall time: 3.95 s


In [53]:
#Store P, Q and vectorizer in pickle file
import pickle
output = open('recommendation.pkl', 'wb')
pickle.dump(P,output)
pickle.dump(Q,output)
pickle.dump(userid_vectorizer,output)
output.close()

## Prediction for input text

In [54]:
words = 'I am intrested in vegeterian restaruents and I want eat falafal oh my god'
test_df= pd.DataFrame([words], columns=['text'])
test_df['text'] = test_df['text'].apply(clean_text)
test_vectors = userid_vectorizer.transform(test_df['text'])
test_v_df = pd.DataFrame(test_vectors.toarray(), index=test_df.index, columns=userid_vectorizer.get_feature_names())

predictItemRating=pd.DataFrame(np.dot(test_v_df.loc[0],Q.T),index=Q.index,columns=['Rating'])
topRecommendations=pd.DataFrame.sort_values(predictItemRating,['Rating'],ascending=[0])[:3]
#topRecommendations


In [55]:
L = topRecommendations.index.values.tolist()
L

['tdcjXyFLMKAsvRhURNOkCg', 'K2_Hmmo5crTYWiT_1sWnfQ', 'uI4YqMarUpchI4I3ZWgOGA']

In [57]:
CSV = pd.read_csv('cuisine_data_final.csv', header=None)
CSV = CSV[[0,1,2]]
CSV
(CSV[2][1].split (","))

["Kashk-e Bademjan (eggplant with whey sauce dip)'",
 "'Boorani Bademjan (eggplant with yogurt sauce dip)'",
 "'Yatimcheh (eggplant with tomato sauce)'",
 "'Naz Khatoon (eggplant and herbs dip with slightly sour taste)'",
 "'Nargesi (with spinach and onions)'"]

## Prediction and Accuracy test on Validation set

In [3]:
f = open('recommendation.pkl', 'rb')
P, Q, userid_vectorizer = pickle.load(f), pickle.load(f), pickle.load(f)

In [14]:
test_df = pd.DataFrame([sentence], columns=['text'])
test_df['text'] = test_df['text'].apply(self.clean_text)
test_vectors = userid_vectorizer.transform(test_df['text'])
test_v_df = pd.DataFrame(test_vectors.toarray(), index=test_df.index,
                         columns=userid_vectorizer.get_feature_names())

pandas.core.series.Series

In [36]:
test_df = X_valid.to_frame()
test_df['text'] = test_df['text'].apply(clean_text)
test_vectors = userid_vectorizer.transform(test_df['text'])
test_v_df = pd.DataFrame(test_vectors.toarray(), index=test_df.index,
                         columns=userid_vectorizer.get_feature_names())


In [49]:
y_pred = []
for key, row in test_v_df.iterrows():
    predictItemRating=pd.DataFrame(np.dot(row,Q.T),index=Q.index,columns=['Rating'])
    topRecommendations=pd.DataFrame.sort_values(predictItemRating,['Rating'],ascending=[0])[:1]
    y_pred.append(topRecommendations.index[0])

In [52]:
#Calculate Accuracy
from sklearn.metrics import accuracy_score
print('Accuracy for validation set is: ',accuracy_score(y_valid, y_pred))

Accuracy for validation set is:  0.684
